# In this notebook, we're going to cover some of the fundamentals concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors and NumPy
* Using @tf.function (a way to speed up your regular python functions)
* Using GPUs with TensorFlow (or TPUs)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Introduction to Tensors

In [2]:
tf.__version__

'2.14.0'

In [3]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
x = tf.constant(0, shape=(3, 3), dtype="int32")
x

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)>

In [5]:
# Check the number of dimensions
x.ndim

2

In [6]:
another_matrix = tf.constant([[1., 2.],
                              [4., 5],
                              [6., 5.]], dtype=tf.float16, shape=(2,3))
another_matrix

<tf.Tensor: shape=(2, 3), dtype=float16, numpy=
array([[1., 2., 4.],
       [5., 6., 5.]], dtype=float16)>

In [7]:
d3_mat = tf.constant([
                       [
                        [1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]
                       ],
                       [
                        [1, 2, 3],
                        [3, 4, 5],
                        [6, 7, 8]
                       ]
                      ])
d3_mat

<tf.Tensor: shape=(2, 3, 3), dtype=int32, numpy=
array([[[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [3, 4, 5],
        [6, 7, 8]]], dtype=int32)>

In [8]:
d3_mat.ndim

3

### Creating tensors with `tf.Variable`

In [9]:
changeable_tensor = tf.Variable([5, 10])
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([ 5, 10], dtype=int32)>

In [10]:
# Modifying the tensor created with .Variable()
changeable_tensor[0].assign(69)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([69, 10], dtype=int32)>

## Creating random tensors

In [11]:
# Create two random (but the same)
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))

random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))

random_1, random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>)

### Shuffle the order of elements in a tensor

In [12]:
# Shuffle a tensor (valuable for when you want to shuffle your data so
# the inherent order does not affect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

# Shuffle our non shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [ 2,  5],
       [10,  7]], dtype=int32)>

In [21]:
tf.random.set_seed(42)
a = tf.random.uniform(shape=(3, 3), minval=1, maxval=10, dtype=tf.int32)
a

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[8, 6, 9],
       [4, 8, 3],
       [8, 8, 8]], dtype=int32)>

### Other ways to create tensors

In [23]:
# Create a tensor of all ones
tf.ones([3, 3], dtype=tf.int32)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=int32)>

In [24]:
# Create a tensor of all zeros
tf.zeros([3, 3], dtype=tf.int32)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)>

### Turn Numpy arrays into tensors

The main difference between Numpy and Tensorflow is that tensors can be run on a GPU way faster

In [27]:
# We can also turn Numpy arrays into arrays
numpy_A = np.arange(1, 25, dtype=np.int32) # create a numpy array
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [44]:
A = tf.constant(numpy_A, shape=(2,3,4))
B = tf.constant(numpy_A, shape=(3, 8))

A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>)

In [45]:
A.shape, B.shape

(TensorShape([2, 3, 4]), TensorShape([3, 8]))

### Getting information from tensors

In [47]:
tf.size(A)

<tf.Tensor: shape=(), dtype=int32, numpy=24>

In [62]:
rank_4_tensor = tf.random.normal(shape=(2, 3, 4, 5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[-0.5590974 , -0.5347214 ,  2.3730333 , -1.5725931 ,
           0.80550563],
         [-0.83387697,  0.3061122 ,  2.2660491 ,  0.2856414 ,
          -1.5536156 ],
         [ 0.37975532,  0.76646256,  0.3611479 ,  0.09653296,
           0.2169556 ],
         [-0.81440705, -0.23623595,  0.49669698, -1.7737728 ,
           0.20886712]],

        [[ 1.0022159 , -0.12915266,  0.16589078,  0.4733353 ,
          -0.834051  ],
         [ 1.013081  ,  0.4107652 ,  0.5531745 ,  1.7808596 ,
          -0.3277541 ],
         [ 0.9474485 ,  0.97951055, -0.46425048,  1.3030937 ,
          -0.24370235],
         [ 0.6692922 ,  0.39855948, -2.4770668 , -0.36922926,
          -0.84287834]],

        [[ 1.8889832 , -0.6198924 ,  1.0777894 ,  1.0240268 ,
           0.26340935],
         [-0.9011545 ,  0.8177133 , -0.27730602,  1.1863395 ,
          -0.11432811],
         [-0.81191176, -0.13922755,  1.8237975 ,  0.09685095,
          -0.1175178

In [49]:
def tensor_info(tnsr):
  print("Datatype of every element:", tnsr.dtype)
  print("Number of dimensions (rank):", tnsr.ndim)
  print("Shape of tensor:", tnsr.shape)
  print("Elements along the 0 axis:", tnsr.shape[0])
  print("Elements along the last axis:", tnsr.shape[-1])
  print("Total number of elements in our tensor:", tf.size(tnsr))
  print("Total number of elements in our tensor:", tf.size(tnsr).numpy())

In [50]:
tensor_info(rank_4_tensor)

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [57]:
# Get the first two elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[ 0.08422458, -0.86090374],
         [ 0.6178192 , -0.33082047]],

        [[ 1.3820479 ,  1.4319172 ],
         [-0.33763257, -0.8959325 ]]],


       [[[-2.3431015 ,  0.7729855 ],
         [ 0.53643376, -0.3456942 ]],

        [[-1.5070348 ,  1.2384809 ],
         [-0.15523385,  0.90184975]]]], dtype=float32)>

In [61]:
rank_4_tensor[:1, :1, :1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=
array([[[[ 0.08422458, -0.86090374,  0.37812304, -0.00519627,
          -0.49453196]]]], dtype=float32)>